In [136]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from pymongo import MongoClient
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import bson

In [137]:
with open('/home/sabinogs/projetos/tcc/reviews/dump/test/reviews.bson', 'rb') as b:
    df = pd.DataFrame(bson.decode_all(b.read()))

In [138]:
comentarios = df[df['requirement_type'].isin(['non-functional requirement', 'other', 'functional requirement'])]
df = df[['reviewText','summary','requirement_type']]
df['pre'] = pd.Series()
comentarios = comentarios[['reviewText','summary','requirement_type']]

In [139]:
def remove_stopwords_sentences(sentences):
    word_without_stop = list()
    for sent in sentences:
        aux = list()
        for word in sent:
            if word not in stopwords.words('english'):
                aux.append(word)
        word_without_stop.append(aux)
    
    return word_without_stop


def remove_stopwords(words):
    return [word for word in words if word not in stopwords.words('english')]

def lemmatize_sentences(sentences):
    lemma = nltk.WordNetLemmatizer()
    lw = list()
    for sent in sentences:
        aux = list()
        for word in sent:
            aux.append(lemma.lemmatize(word))
        lw.append(aux)
    
    return lw

def lemmatize_it(words):
    lemma = nltk.WordNetLemmatizer()
    return [lemma.lemmatize(word) for word in words]


def remove_pon(doc):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(doc)

def _stem_it(doc):
    stem = PorterStemmer()
    return [stem.stem(word) for word in doc]


In [197]:
pre = comentarios['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it)
pre_summary = comentarios['summary'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it)

In [198]:
pre = pre + pre_summary

In [209]:
comentarios['pre'] = pre
comentarios['pre'] = comentarios['pre'].apply(lambda x: ' '.join(x))
comentarios.head()

,reviewText,summary,requirement_type,pre
0,How dissapointing..downloaded and found it doe...,Extreme Dissappointment,non-functional requirement,how dissapoint download and found it doe not a...
1,This is great for kids my two year old son lov...,great,other,thi is great for kid my two year old son love ...
2,"Loves the song, so he really couldn't wait to ...",Really cute,non-functional requirement,love the song so he realli couldn t wait to pl...
3,My three year old Plays this game the most he ...,Five little monkeys,other,My three year old play thi game the most he lo...
4,As a Speech language pathology Assistant I hav...,My patients request this app everytime they se...,other,As a speech languag patholog assist I have a v...


In [210]:
c = comentarios[comentarios['requirement_type'] != 'other']
c['pre'] = c['pre'].apply(lambda x: ' '.join(x))

/home/sabinogs/.virtualenvs/explore/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [211]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


Para maior referencia, olhe os links acima

In [212]:
X = c['pre'].tolist()
y = c['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [213]:
text_classifier = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

In [214]:
text_classifier.fit(X_train,y_train)
predicted = text_classifier.predict(X_test)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [215]:
print(classification_report(y_test,predicted))

                            precision    recall  f1-score   support

    functional requirement       0.60      0.98      0.74       176
non-functional requirement       0.75      0.10      0.17       126

                 micro avg       0.61      0.61      0.61       302
                 macro avg       0.68      0.54      0.46       302
              weighted avg       0.66      0.61      0.50       302



# Um Classificador para identificar todas as categorias (Experimento 1)

In [221]:
comentarios['requirement_type'].value_counts()

other                         2295
functional requirement         596
non-functional requirement     410
Name: requirement_type, dtype: int64

In [219]:
X = comentarios['pre'].tolist()
y = comentarios['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

text_classifier_exp1 = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

text_classifier_exp1.fit(X_train,y_train)
predicted_exp1 = text_classifier_exp1.predict(X_test)
print(classification_report(y_test, predicted_exp1))

                            precision    recall  f1-score   support

    functional requirement       0.00      0.00      0.00       177
non-functional requirement       0.00      0.00      0.00       120
                     other       0.70      1.00      0.82       694

                 micro avg       0.70      0.70      0.70       991
                 macro avg       0.23      0.33      0.27       991
              weighted avg       0.49      0.70      0.58       991



/home/sabinogs/.virtualenvs/explore/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Nessa seção criarei um classificador para cada Requisito



Isso se dá pela necessidade de aumentar o desempenho do classificador. Para isso, será criado 3 classificadores que: 

1. Identificará se um comentário é do tipo "Funcional" ou não
2. Identificará se um comentário é do tipo "Não Funcional" ou não
3. Identificará se um comentário é do tipo "Outro" ou não


Observe o uso do `DataFrame.copy()`. Isso deve-se ao fato de que ao realizar uma atribuição de dataframe `novoDF = velho` copiamos apenas a referência e isso implicará que mudar uma coisa no `novoDF` muda tbm no `velho`. 



Ref: 

1. https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.copy.html

In [167]:

nao_funcionais = comentarios.copy(deep=True)
outros = comentarios.copy(deep=True)

In [168]:
def transform_dataset(documento, requirement_type):
    if documento == requirement_type:
        return 'sim'
    else:
        return 'nao'

In [169]:



nao_funcionais['requirement_type'] = nao_funcionais['requirement_type'].apply(
    lambda x: transform_dataset(x,'non-functional requirement')
)


outros['requirement_type'] = outros['requirement_type'].apply(
    lambda x: transform_dataset(x,'other')
)


# Experimento 2

## Dataset e Classificador para FUNCIONAIS

In [223]:
funcionais = comentarios.copy(deep=True)
funcionais['requirement_type'] = funcionais['requirement_type'].apply(
    lambda x: transform_dataset(x,'functional requirement')
)

In [231]:
funcionais['requirement_type'].value_counts()

nao    2705
sim     596
Name: requirement_type, dtype: int64

In [224]:
_pre_review = funcionais['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)
_pre_summary = funcionais['summary'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)

pre = _pre_review + _pre_summary
funcionais['pre'] = pre
funcionais['pre'] = funcionais['pre'].apply(lambda x: ' '.join(x))

X_funcionais = funcionais['pre'].tolist()
y_funcionais = funcionais['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X_funcionais, y_funcionais, test_size=0.3, random_state=42)

text_classifier_funcionais = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

text_classifier_funcionais.fit(X_train, y_train)
predicted_funcionais = text_classifier_funcionais.predict(X_test)
print(classification_report(y_test, predicted_funcionais))

              precision    recall  f1-score   support

         nao       0.82      1.00      0.90       814
         sim       0.00      0.00      0.00       177

   micro avg       0.82      0.82      0.82       991
   macro avg       0.41      0.50      0.45       991
weighted avg       0.67      0.82      0.74       991



/home/sabinogs/.virtualenvs/explore/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Dataset e Classificador para NAO FUNCIONAIS

In [237]:
nao_funcionais = comentarios.copy(deep=True)
nao_funcionais['requirement_type'] = nao_funcionais['requirement_type'].apply(
    lambda x: transform_dataset(x,'non-functional requirement')
)

In [238]:
nao_funcionais['requirement_type'].value_counts()

nao    2891
sim     410
Name: requirement_type, dtype: int64

In [240]:
_pre_review = nao_funcionais['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)
_pre_summary = nao_funcionais['summary'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)

pre = _pre_review + _pre_summary
nao_funcionais['pre'] = pre
nao_funcionais['pre'] = nao_funcionais['pre'].apply(lambda x: ' '.join(x))

X_nao_funcionais = nao_funcionais['pre'].tolist()
y_nao_funcionais = nao_funcionais['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X_nao_funcionais, y_nao_funcionais, test_size=0.3, random_state=42)

text_classifier_nao_funcionais = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

text_classifier_nao_funcionais.fit(X_train, y_train)
predicted_nao_funcionais = text_classifier_nao_funcionais.predict(X_test)
print(classification_report(y_test, predicted_nao_funcionais))

              precision    recall  f1-score   support

         nao       0.88      1.00      0.94       871
         sim       0.00      0.00      0.00       120

   micro avg       0.88      0.88      0.88       991
   macro avg       0.44      0.50      0.47       991
weighted avg       0.77      0.88      0.82       991



/home/sabinogs/.virtualenvs/explore/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Experimento 3

## Criando datasets balanceados

### Funcionais

In [241]:
index_array = funcionais[funcionais['requirement_type'] == 'nao'].index
funcionais_to_be_removed = np.random.choice(index_array,int(3*len(index_array)/4))

funcionais_balanceado = funcionais.drop(funcionais_to_be_removed)
funcionais_balanceado['requirement_type'].value_counts()

nao    1286
sim     596
Name: requirement_type, dtype: int64

### Não funcionais

In [243]:
index_array = nao_funcionais[nao_funcionais['requirement_type'] == 'nao'].index
nao_funcionais_to_be_removed = np.random.choice(index_array,int(3*len(index_array)/4))

nao_funcionais_balanceado = nao_funcionais.drop(nao_funcionais_to_be_removed)
nao_funcionais_balanceado['requirement_type'].value_counts()

nao    1344
sim     410
Name: requirement_type, dtype: int64

# Criando os classificadores balanceados

### Funcionais

In [244]:
_pre_review = funcionais_balanceado['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)
_pre_summary = funcionais_balanceado['summary'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)

pre = _pre_review + _pre_summary
funcionais_balanceado['pre'] = pre
funcionais_balanceado['pre'] = funcionais_balanceado['pre'].apply(lambda x: ' '.join(x))

X_funcionais = funcionais_balanceado['pre'].tolist()
y_funcionais = funcionais_balanceado['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X_funcionais, y_funcionais, test_size=0.3, random_state=42)

text_classifier_funcionais_balanceados = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

text_classifier_funcionais_balanceados.fit(X_train, y_train)
predicted_funcionais_balanceados = text_classifier_funcionais_balanceados.predict(X_test)
print(classification_report(y_test, predicted_funcionais_balanceados))

              precision    recall  f1-score   support

         nao       0.70      0.99      0.82       392
         sim       0.77      0.06      0.11       173

   micro avg       0.71      0.71      0.71       565
   macro avg       0.74      0.53      0.47       565
weighted avg       0.72      0.71      0.60       565



### Não funcionais

In [245]:
_pre_review = nao_funcionais_balanceado['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)
_pre_summary = nao_funcionais_balanceado['summary'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)

pre = _pre_review + _pre_summary
nao_funcionais_balanceado['pre'] = pre
nao_funcionais_balanceado['pre'] = nao_funcionais_balanceado['pre'].apply(lambda x: ' '.join(x))

X_nao_funcionais = nao_funcionais_balanceado['pre'].tolist()
y_nao_funcionais = nao_funcionais_balanceado['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X_nao_funcionais, y_nao_funcionais, test_size=0.3, random_state=42)

text_classifier_nao_funcionais_balanceados = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

text_classifier_nao_funcionais_balanceados.fit(X_train, y_train)
predicted_nao_funcionais_balanceados = text_classifier_nao_funcionais_balanceados.predict(X_test)
print(classification_report(y_test, predicted_nao_funcionais_balanceados))

              precision    recall  f1-score   support

         nao       0.75      1.00      0.86       397
         sim       1.00      0.01      0.02       130

   micro avg       0.76      0.76      0.76       527
   macro avg       0.88      0.50      0.44       527
weighted avg       0.82      0.76      0.65       527



In [184]:
index_array = funcionais[funcionais['requirement_type'] == 'nao'].index
to_be_removed = np.random.choice(index_array,int(3*len(index_array)/4))

_f = funcionais.drop(to_be_removed)
_f['requirement_type'].value_counts()
# len(funcionais[funcionais['requirement_type'] == 'nao'])
# len(to_be_removed)

nao    1290
sim     596
Name: requirement_type, dtype: int64


## Classificador de Requisitos Funcionais

In [185]:
_f['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it)

4       [As, a, speech, languag, patholog, assist, I, ...
5       [thought, it, would, be, the, book, to, read, ...
7       [We, love, these, monkey, s, and, all, the, co...
8       [I, found, thi, at, a, perfect, time, sinc, my...
9       [My, granddaught, an, 18, month, old, love, th...
10      [I, like, the, fact, that, I, can, purchas, mu...
11      [I, watch, my, great, grandson, 4, day, a, wee...
12      [It, wont, download, my, playlist, offlin, any...
13      [I, have, a, kindl, fire, first, gen, and, thi...
15      [I, am, so, disappoint, that, I, actual, paid,...
16      [No, matter, what, kind, of, music, I, want, t...
17      [you, cant, use, thi, unless, you, have, a, pa...
18      [is, the, music, I, like, non, commerci, so, I...
21      [love, that, I, can, type, ani, song, in, and,...
22      [not, worth, the, download, what, a, wast, of,...
23      [My, granddaught, realli, love, thi, she, will...
25      [onli, issu, is, new, music, doesnt, alway, sh...
26      [great

In [190]:
_pre_review = _f['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)
_pre_summary = _f['summary'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)

pre = _pre_review + _pre_summary
_f['pre'] = pre
_f['pre'] = _f['pre'].apply(lambda x: ' '.join(x))

X_funcionais = _f['pre'].tolist()
y_funcionais = _f['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X_funcionais, y_funcionais, test_size=0.3, random_state=42)

text_classifier_funcionais = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

text_classifier_funcionais.fit(X_train, y_train)
predicted_funcionais = text_classifier_funcionais.predict(X_test)
print(classification_report(y_test, predicted_funcionais))

              precision    recall  f1-score   support

         nao       0.69      1.00      0.82       383
         sim       0.92      0.07      0.12       183

   micro avg       0.70      0.70      0.70       566
   macro avg       0.81      0.53      0.47       566
weighted avg       0.77      0.70      0.59       566



In [191]:
_pre_review = funcionais['reviewText'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)
_pre_summary = funcionais['summary'].apply(remove_pon).apply(_stem_it).apply(lemmatize_it).apply(remove_stopwords)

pre = _pre_review + _pre_summary
funcionais['pre'] = pre
funcionais['pre'] = funcionais['pre'].apply(lambda x: ' '.join(x))

X_funcionais = funcionais['pre'].tolist()
y_funcionais = funcionais['requirement_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X_funcionais, y_funcionais, test_size=0.3, random_state=42)

text_classifier_funcionais = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

text_classifier_funcionais.fit(X_train, y_train)
predicted_funcionais = text_classifier_funcionais.predict(X_test)
print(classification_report(y_test, predicted_funcionais))

              precision    recall  f1-score   support

         nao       0.82      1.00      0.90       814
         sim       0.00      0.00      0.00       177

   micro avg       0.82      0.82      0.82       991
   macro avg       0.41      0.50      0.45       991
weighted avg       0.67      0.82      0.74       991



/home/sabinogs/.virtualenvs/explore/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
